In [69]:
import requests
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import re
import csv
import time

In [6]:
def scrape_game_names(page_num):
    url = 'https://boardgamegeek.com/browse/boardgame/page/{}?sort=numvoters&sortdir=desc'.format(page_num)
    r = requests.get(url)
    html = r.text
    xpath_names = "//tr/td[3]/div/a/text()"
    xpath_links = "//tr/td[3]/div/a/@href"
    
    names = Selector(text=html).xpath(xpath_names).extract()
    links = Selector(text=html).xpath(xpath_links).extract()
    
    return names, links

In [25]:
def get_game_ids(links):
    search = re.compile('\/(\d+)\/').search
    ids = [m.group(1) for m in map(search, links) if m]
    return ids

In [48]:
def save_game_csv(page_num, names, links, ids):
    rows = zip(ids, names, links)
    
    with open('games/page_{}'.format(page_num), 'w') as f:
        writer = csv.writer(f)
        for row in rows:
            writer.writerow(row)

In [73]:
for page_num in range(1,101):
    names, links = scrape_game_names(page_num)
    ids = get_game_ids(links)
    save_game_csv(page_num, names, links, ids)
    time.sleep(2)